In [1]:
import tensorflow as tf
import keras
import numpy as np
import string
from keras.preprocessing.text import Tokenizer
#!pip install pronouncing
import pronouncing
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

2022-12-09 11:56:14.080717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 11:56:14.161341: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-09 11:56:14.541927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/

In [2]:
#PREPROCESSING


training_file_path = "poems.txt"

f = open(training_file_path, "r", encoding="utf-8")
content = f.read()# read all contents
f.close()  # close the file when you're done

#(word,word,word,word,etc.)
content = content.lower()
content_no_punctuation = ""
i = 0

while i < (len(content)):
  if content[i] not in string.punctuation:
    content_no_punctuation += content[i]
  i += 1

content = content_no_punctuation

lines = content.splitlines()
data = []
for l in lines:
  sentence = l.split()
  sentence.append('\n')
  data.append(sentence)
  

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
encoded = tokenizer.texts_to_sequences(data)


flatten_list = [j for sub in encoded for j in sub]
vocab = sorted(set(tokenizer.word_docs))

In [3]:
ids_dataset = tf.data.Dataset.from_tensor_slices(flatten_list)

seq_length = 200

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

2022-12-09 11:56:49.514913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-09 11:56:49.518040: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-09 11:56:49.518136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-09 11:56:49.518432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [4]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [5]:
# Batch size
BATCH_SIZE = 32

dataset = (
    dataset
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

vocab_size = len(tokenizer.word_index) + 1
vocab_size1 = len(set(flatten_list))

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [6]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [7]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [8]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
model.compile(optimizer='adam', loss=loss)

In [10]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/100


2022-12-09 11:56:51.574413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-12-09 11:56:52.056972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-09 11:56:52.125641: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fed293b97a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-09 11:56:52.125668: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2022-12-09 11:56:52.128596: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-09 11:56:52.171815: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is ol

 1/27 [>.............................] - ETA: 1:07 - loss: 9.4206

2022-12-09 11:56:52.401740: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-12-09 11:56:52.547605: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-12-09 11:56:52.552141: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

 2/27 [=>............................] - ETA: 8s - loss: 9.4169  

...

2022-12-09 11:57:01.934281: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


26/27 [===========================>..] - ETA: 0s - loss: 7.3048

2022-12-09 11:57:02.327089: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


27/27 [==============================] - 13s 390ms/step - loss: 7.2853
Epoch 2/100
27/27 [==============================] - 10s 380ms/step - loss: 6.5882
Epoch 3/100
27/27 [==============================] - 10s 355ms/step - loss: 6.4949
Epoch 4/100
27/27 [==============================] - 10s 375ms/step - loss: 6.2135
Epoch 5/100
27/27 [==============================] - 10s 374ms/step - loss: 6.0633
Epoch 6/100
27/27 [==============================] - 10s 378ms/step - loss: 5.9580
Epoch 7/100
27/27 [==============================] - 11s 391ms/step - loss: 5.8652
Epoch 8/100
27/27 [==============================] - 10s 373ms/step - loss: 5.7706
Epoch 9/100
27/27 [==============================] - 10s 365ms/step - loss: 5.6851
Epoch 10/100
27/27 [==============================] - 10s 355ms/step - loss: 5.5990
Epoch 11/100
27/27 [==============================] - 10s 355ms/step - loss: 5.5131
Epoch 12/100
27/27 [==============================] - 10s 355ms/step - loss: 5.4264
Epoch 13/100


27/27 [==============================] - 10s 369ms/step - loss: 0.5123
Epoch 99/100
27/27 [==============================] - 10s 367ms/step - loss: 0.4796
Epoch 100/100
27/27 [==============================] - 10s 358ms/step - loss: 0.4496


In [11]:
class OneStep(tf.keras.Model):
  def __init__(self, model, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model

  #@tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    
    temp = tokenizer.texts_to_sequences(inputs)
    if len(temp[0]) == 0:
      temp = [[1]]
    input_ids = np.array(temp)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    #predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = tokenizer.sequences_to_texts([predicted_ids.numpy()])
    
    # Return the characters and model state.
    return predicted_chars, states

In [12]:
def get_rhyme_with_max_embedding(base_word, context_words):
  
  all_rhyme_words = pronouncing.rhymes(base_word)

  best_count = float('-inf')
  best_word = ""
  for r in all_rhyme_words:
    count = 0
    seen_content = 1
    for c in context_words:
      if r in glove_vectors and c[0] in glove_vectors:
        count += glove_vectors.similarity(r, c[0])
        seen_content += 1
    
    count = count / seen_content
    
    
    if count >= best_count:
      best_word = r
      best_count = count

  return best_word

In [13]:
next_char = ['‘twas']
rhyme_scheme = 'ABBCBB'

one_step_model = OneStep(model)
states = None

result = [next_char]

breaks = rhyme_scheme.count(" ")
lines = len(rhyme_scheme) - breaks
n = 0
prev_char = []


while n < lines:
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  
  if next_char == ['\n']:
    if prev_char != ['\n']:
      result.append(next_char)
      n += 1
  else:
    result.append(next_char)
    
  prev_char = next_char


In [14]:
line_count = 0
rhymes = {}

temp_scheme = rhyme_scheme.replace(" ", "")
i = 0
start_index = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    rhyme_index = temp_scheme[line_count]
    if rhyme_index in rhymes:
      base_rhyme_word = rhymes[rhyme_index]
      embedding_words = result[start_index:i-1]
      rhymed_word = get_rhyme_with_max_embedding(base_rhyme_word[0], embedding_words)
      result[i-1] = [rhymed_word]
      rhymes[rhyme_index] = result[i-1]
      
    else:
      rhymes[rhyme_index] = result[i-1]
    
    start_index = i + 1
    line_count += 1
    
    
start_index = 0
line_count = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    if rhyme_scheme[line_count] == " ":
        result[start_index - 1] = ['\n\n']
        
    line_count += 1
    start_index = i + 1

In [15]:
poem = " "
for l in result:
  poem += l[0]
  poem += " "
print(poem)

 ‘twas in all their ancient faces like gifts 
 and may they all do their gifts 
 with all untold shifts 
 art thou really are the children 
 as god on their joyous gifts 
 blow thou little white christmas shifts 
 


In [16]:
model.save('model_poems')
#model = keras.load_model('model_poems')

INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 512]), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fefac6fdf70>, 140667696630352), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512, 3072), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fefac711670>, 140667696631120), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024, 3072), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fefac6c0ee0>, 140667696095568), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(2, 3072), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fefac6f6e80>, 140667696240480), {}).
INFO:tensorflow:Unsupported signature for ser

INFO:tensorflow:Assets written to: model_poems/assets


INFO:tensorflow:Assets written to: model_poems/assets
